# Analyzing Geospatial Data

## Introduction

Click the **Open in Colab** button below to open this notebook in Google Colab:

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geebook/blob/master/chapters/06_data_analysis.ipynb)

## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```

```bash
jupyter lab
```

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

## Reducer

### List reductions

In [ ]:
values = ee.List.sequence(1, 10)
print(values.getInfo())

In [ ]:
count = values.reduce(ee.Reducer.count())
print(count.getInfo())  # 10

In [ ]:
min_value = values.reduce(ee.Reducer.min())
print(min_value.getInfo())  # 1

In [ ]:
max_value = values.reduce(ee.Reducer.max())
print(max_value.getInfo())  # 10

In [ ]:
min_max_value = values.reduce(ee.Reducer.minMax())
print(min_max_value.getInfo())

In [ ]:
mean_value = values.reduce(ee.Reducer.mean())
print(mean_value.getInfo())  # 5.5

In [ ]:
median_value = values.reduce(ee.Reducer.median())
print(median_value.getInfo())  # 5.5

In [ ]:
sum_value = values.reduce(ee.Reducer.sum())
print(sum_value.getInfo())  # 55

In [ ]:
std_value = values.reduce(ee.Reducer.stdDev())
print(std_value.getInfo())  # 2.8723

### ImageCollection reductions

In [ ]:
Map = geemap.Map()

# Load an image collection, filtered so it's not too much data.
collection = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
    .filterDate('2021-01-01', '2021-12-31')
    .filter(ee.Filter.eq('WRS_PATH', 44))
    .filter(ee.Filter.eq('WRS_ROW', 34))
)

# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
median = collection.reduce(ee.Reducer.median())

# The output is an Image.  Add it to the map.
vis_param = {'bands': ['B5_median', 'B4_median', 'B3_median'], 'gamma': 2}
Map.setCenter(-122.3355, 37.7924, 8)
Map.addLayer(median, vis_param)
Map

In [ ]:
median = collection.median()
print(median.bandNames().getInfo())

### Image reductions

In [ ]:
import geemap

Map = geemap.Map()

# Load an image and select some bands of interest.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318').select(['B4', 'B3', 'B2'])

# Reduce the image to get a one-band maximum value image.
maxValue = image.reduce(ee.Reducer.max())

# Display the result.
Map.centerObject(image, 8)
Map.addLayer(image, {}, 'Original image')
Map.addLayer(maxValue, {'max': 13000}, 'Maximum value image')
Map

### FeatureCollection reductions

In [ ]:
Map = geemap.Map()

# Load US cenus data as a FeatureCollection.
census = ee.FeatureCollection('TIGER/2010/Blocks')

# Filter the collection to include only Benton County, OR.
benton = census.filter(
    ee.Filter.And(ee.Filter.eq('statefp10', '41'), ee.Filter.eq('countyfp10', '003'))
)

# Display Benton County cenus blocks.
Map.setCenter(-123.27, 44.57, 13)
Map.addLayer(benton)
Map

In [ ]:
# Compute sums of the specified properties.
properties = ['pop10', 'housing10']
sums = benton.filter(ee.Filter.notNull(properties)).reduceColumns(
    **{'reducer': ee.Reducer.sum().repeat(2), 'selectors': properties}
)

# Print the resultant Dictionary.
print(sums.getInfo())

In [ ]:
print(benton.aggregate_sum('pop10').getInfo())  # 85579
print(benton.aggregate_sum('housing10').getInfo())  # 36245

In [ ]:
benton.aggregate_stats('pop10').getInfo()

## Image descriptive statistics

In [ ]:
Map = geemap.Map()

centroid = ee.Geometry.Point([-122.4439, 37.7538])
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(centroid).first()
vis = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(centroid, 8)
Map.addLayer(image, vis, "Landsat-8")
Map

In [ ]:
image.propertyNames().getInfo()

In [ ]:
image.get('CLOUD_COVER').getInfo()  # 0.05

In [ ]:
props = geemap.image_props(image)
props.getInfo()

In [ ]:
stats = geemap.image_stats(image, scale=30)
stats.getInfo()

## Zonal statistics

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NASA SRTM
dem = ee.Image('USGS/SRTMGL1_003')
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add 5-year Landsat TOA composite
landsat = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(landsat, landsat_vis, "Landsat", False)

# Add US Census States
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')
Map

In [ ]:
out_dem_stats = 'dem_stats.csv'
geemap.zonal_stats(
    dem, states, out_dem_stats, statistics_type='MEAN', scale=1000, return_fc=False
)

In [ ]:
out_landsat_stats = 'landsat_stats.csv'
geemap.zonal_stats(
    landsat,
    states,
    out_landsat_stats,
    statistics_type='MEAN',
    scale=1000,
    return_fc=False,
)

## Zonal statistics by group

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NLCD data
dataset = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = dataset.select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2019')

# Add US census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')

# Add NLCD legend
Map.add_legend(title='NLCD Land Cover', builtin_legend='NLCD')
Map

In [ ]:
nlcd_stats = 'nlcd_stats.csv'

geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type='SUM',
    denominator=1000000,
    decimal_places=2,
)

In [ ]:
nlcd_stats = 'nlcd_stats_pct.csv'

geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type='PERCENTAGE',
    denominator=1000000,
    decimal_places=2,
)

## Zonal statistics with two images

In [ ]:
import ee
import geemap
import geemap.colormaps as cm

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map

In [ ]:
dem = ee.Image('USGS/3DEP/10m')
vis = {'min': 0, 'max': 4000, 'palette': cm.palettes.dem}

In [ ]:
Map.addLayer(dem, vis, 'DEM')

In [ ]:
landcover = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select('landcover')

In [ ]:
Map.addLayer(landcover, {}, 'NLCD 2019')
Map.add_legend(builtin_legend='NLCD')

In [ ]:
stats = geemap.image_stats_by_zone(dem, landcover, reducer='MEAN')
stats

In [ ]:
stats.to_csv('mean.csv', index=False)

In [ ]:
geemap.image_stats_by_zone(dem, landcover, out_csv="std.csv", reducer='STD')

## Creating coordinate grids

In [ ]:
Map = geemap.Map()

In [ ]:
lat_grid = geemap.latitude_grid(step=5.0, west=-180, east=180, south=-85, north=85)

In [ ]:
Map.addLayer(lat_grid, {}, 'Latitude Grid')

In [ ]:
Map

In [ ]:
df = geemap.ee_to_df(lat_grid)
df

In [ ]:
lon_grid = geemap.longitude_grid(step=5.0, west=-180, east=180, south=-85, north=85)

In [ ]:
Map = geemap.Map()
Map.addLayer(lon_grid, {}, 'Longitude Grid')
Map

In [ ]:
grid = geemap.latlon_grid(
    lat_step=10, lon_step=10, west=-180, east=180, south=-85, north=85
)

In [ ]:
Map = geemap.Map()
Map.addLayer(grid, {}, 'Coordinate Grid')
Map

## Creating fishnets

In [ ]:
Map = geemap.Map()
Map

In [ ]:
data = Map.user_roi

if data is None:
    data = ee.Geometry.BBox(-112.8089, 33.7306, -88.5951, 46.6244)
    Map.addLayer(data, {}, 'ROI')
    Map.user_roi = None

Map.centerObject(data)

In [ ]:
fishnet = geemap.fishnet(data, h_interval=2.0, v_interval=2.0, delta=1)

In [ ]:
Map.addLayer(fishnet, {}, 'Fishnet 1')

In [ ]:
data = Map.user_roi

if data is None:
    data = ee.Geometry.Polygon(
        [
            [
                [-64.602356, -1.127399],
                [-68.821106, -12.625598],
                [-60.647278, -22.498601],
                [-47.815247, -21.111406],
                [-43.860168, -8.913564],
                [-54.582825, -0.775886],
                [-60.823059, 0.454555],
                [-64.602356, -1.127399],
            ]
        ]
    )
    Map.addLayer(data, {}, 'ROI2')

Map.centerObject(data)
Map

In [ ]:
fishnet = geemap.fishnet(data, rows=6, cols=8, delta=1)

In [ ]:
Map.addLayer(fishnet, {}, 'Fishnet 2')

## Sankey diagrams

In [ ]:
import sankee

sankee.datasets.LCMS_LC.sankify(
    years=[1990, 2000, 2010, 2020],
    region=ee.Geometry.Point([-122.192688, 46.25917]).buffer(2000),
    max_classes=3,
    title="Mount St. Helens Recovery",
)

In [ ]:
Map = geemap.Map(height=650)
Map

## Mapping available imagery

In [ ]:
import geemap.colormaps as cm

In [ ]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
image = geemap.image_count(
    collection, region=None, start_date='2021-01-01', end_date='2022-01-01', clip=False
)

In [ ]:
Map = geemap.Map()
vis = {'min': 0, 'max': 60, 'palette': cm.palettes.coolwarm}
Map.addLayer(image, vis, 'Landsat 8 Image Count')

countries = ee.FeatureCollection('users/giswqs/public/countries')
style = {"color": "00000088", "width": 1, "fillColor": "00000000"}
Map.addLayer(countries.style(**style), {}, "Countries")
Map.add_colorbar(vis, label='Landsat 8 Image Count')

Map

## Using Landsat 9

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()

In [ ]:
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
print(collection.size().getInfo())

In [ ]:
median = collection.median()

In [ ]:
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [ ]:
dataset = apply_scale_factors(median)

In [ ]:
vis_natural = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

vis_nir = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

In [ ]:
Map.addLayer(dataset, vis_natural, 'True color (432)')
Map.addLayer(dataset, vis_nir, 'Color infrared (543)')
Map

In [ ]:
vis_params = [
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B7', 'SR_B6', 'SR_B4'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B6', 'SR_B5', 'SR_B2'], 'min': 0, 'max': 0.3},
]

In [ ]:
labels = [
    'Natural Color (4, 3, 2)',
    'Color Infrared (5, 4, 3)',
    'Short-Wave Infrared (7, 6 4)',
    'Agriculture (6, 5, 2)',
]

In [ ]:
geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=[40, -100],
    zoom=4,
    ee_objects=[dataset],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

In [ ]:
landsat8 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_015043_20130402')
landsat9 = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_015043_20211231')

In [ ]:
landsat8 = apply_scale_factors(landsat8)
landsat9 = apply_scale_factors(landsat9)

In [ ]:
left_layer = geemap.ee_tile_layer(landsat8, vis_natural, 'Landsat 8')
right_layer = geemap.ee_tile_layer(landsat9, vis_natural, 'Landsat 9')

In [ ]:
Map = geemap.Map()
Map.split_map(left_layer, right_layer)
Map

## Interactive region reduction

### Import libraries

In [ ]:
import os
import ee
import geemap

### Create an interactive map

In [ ]:
m = geemap.Map()

### Add add to the map

In [ ]:
collection = (
    ee.ImageCollection('MODIS/006/MOD13A2')
    .filterDate('2015-01-01', '2019-12-31')
    .select('NDVI')
)

# Convert the image collection to an image.
image = collection.toBands()

ndvi_vis = {
    'min': 0.0,
    'max': 9000.0,
    'palette': [
        'FFFFFF',
        'CE7E45',
        'DF923D',
        'F1B555',
        'FCD163',
        '99B718',
        '74A901',
        '66A000',
        '529400',
        '3E8601',
        '207401',
        '056201',
        '004C00',
        '023B01',
        '012E01',
        '011D01',
        '011301',
    ],
}

m.addLayer(image, {}, 'MODIS NDVI Time-series')
m.addLayer(image.select(0), ndvi_vis, 'MODIS NDVI VIS')

m

### Set reducer

In [ ]:
m.set_plot_options(add_marker_cluster=True, marker=None)
m.roi_reducer = ee.Reducer.mean()

### Export data

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_csv = os.path.join(out_dir, 'points.csv')
out_shp = os.path.join(out_dir, 'ndvi.shp')
m.extract_values_to_points(out_shp)

## Extracting values to points

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine layers to Map
Map.addLayer(
    landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200}, 'Landsat 7'
)
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 1)

In [ ]:
work_dir = os.path.expanduser('~/Downloads')
in_shp = os.path.join(work_dir, 'us_cities.shp')
if not os.path.exists(in_shp):
    data_url = 'https://github.com/giswqs/data/raw/main/us/us_cities.zip'
    geemap.download_from_url(data_url, out_dir=work_dir)

In [ ]:
in_fc = geemap.shp_to_ee(in_shp)
Map.addLayer(in_fc, {}, 'Cities')

In [ ]:
out_shp = os.path.join(work_dir, 'dem.shp')
geemap.extract_values_to_points(in_fc, dem, out_shp)

In [ ]:
out_csv = os.path.join(work_dir, 'landsat.csv')
geemap.extract_values_to_points(in_fc, landsat7, out_csv)

## Extracting pixels along transect

In [ ]:
from bqplot import pyplot as plt

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("TERRAIN")
Map

In [ ]:
image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTM DEM', True, 0.5)

In [ ]:
# Use the drawing tool to draw any line on the map.
line = Map.user_roi
if line is None:
    line = ee.Geometry.LineString(
        [[-120.223279, 36.314849], [-118.926969, 36.712192], [-117.202217, 36.756215]]
    )
    Map.addLayer(line, {}, "ROI")
Map.centerObject(line)

In [ ]:
line.getInfo()

In [ ]:
reducer = 'mean'  # Any ee.Reducer, e.g., mean, median, min, max, stdDev
transect = geemap.extract_transect(
    image, line, n_segments=100, reducer=reducer, to_pandas=True
)

In [ ]:
transect

In [ ]:
transect.to_csv('transect.csv')

In [ ]:
fig = plt.figure()
plt.plot(transect['distance'], transect[reducer])
plt.xlabel('Distance')
plt.ylabel("Elevation")
plt.show()

## Unsupervised classification

### Unsupervised classification algorithms available in Earth Engine

### Step-by-step tutorial

#### Import libraries

In [ ]:
import ee
import geemap

#### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

#### Add data to the map

In [ ]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2019-01-01', '2019-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

#### Check image properties

In [ ]:
props = geemap.image_props(image)
props.getInfo()

In [ ]:
props.get('IMAGE_DATE').getInfo()

In [ ]:
props.get('CLOUD_COVER').getInfo()

#### Make training dataset

In [ ]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [ ]:
# Make the training dataset.
training = image.sample(
    **{
        #     'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'training', False)
Map

#### Train the clusterer

In [ ]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

#### Classify the image

In [ ]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

#### Label the clusters

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

#### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

#### Export the result

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

In [ ]:
geemap.ee_export_image(result, filename=out_file, scale=90)

In [ ]:
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=90
)

## Supervised classification

### Supervised classification algorithms available in Earth Engine

### Step-by-step tutorial

#### Import libraries

In [ ]:
import ee
import geemap

#### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

#### Add data to the map

In [ ]:
point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2016-01-01', '2016-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

#### Check image properties

In [ ]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

In [ ]:
image.get('CLOUD_COVER').getInfo()

#### Make training dataset

In [ ]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [ ]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')
Map

In [ ]:
# Make the training dataset.
points = nlcd.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, 'training', False)

In [ ]:
print(points.size().getInfo())

In [ ]:
print(points.first().getInfo())

#### Train the classifier

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [ ]:
print(training.first().getInfo())

#### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

#### Render categorical map

In [ ]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_values

In [ ]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
class_palette

In [ ]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Land cover')
Map

#### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

#### Add a legend to the map

In [ ]:
Map.add_legend(builtin_legend='NLCD')
Map

#### Export the result

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

In [ ]:
geemap.ee_export_image_to_drive(
    landcover, description='landcover', folder='export', scale=900
)

## Accuracy assessment

### Supervised classification algorithms available in Earth Engine

### Step-by-step tutorial

#### Import libraries

In [ ]:
import ee
import geemap

#### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

#### Add data to the map

In [ ]:
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(NLCD2016, {}, 'NLCD 2016')

In [ ]:
NLCD_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
Map.addLayer(NLCD_metadata, {}, 'NLCD Metadata')

In [ ]:
# point = ee.Geometry.Point([-122.4439, 37.7538])  # Sanfrancisco, CA
# point = ee.Geometry.Point([-83.9293, 36.0526])   # Knoxville, TN
point = ee.Geometry.Point([-88.3070, 41.7471])  # Chicago, IL

In [ ]:
metadata = NLCD_metadata.filterBounds(point).first()
region = metadata.geometry()

In [ ]:
metadata.get('2016on_bas').getInfo()

In [ ]:
doy = metadata.get('2016on_bas').getInfo().replace('LC08_', '')
doy

In [ ]:
ee.Date.parse('YYYYDDD', doy).format('YYYY-MM-dd').getInfo()

In [ ]:
start_date = ee.Date.parse('YYYYDDD', doy)
end_date = start_date.advance(1, 'day')

In [ ]:
image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate(start_date, end_date)
    .first()
    .select('B[1-7]')
    .clip(region)
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

In [ ]:
nlcd_raw = NLCD2016.clip(region)
Map.addLayer(nlcd_raw, {}, 'NLCD')

#### Prepare for consecutive class labels

In [ ]:
raw_class_values = nlcd_raw.get('landcover_class_values').getInfo()
print(raw_class_values)

In [ ]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

In [ ]:
class_palette = nlcd_raw.get('landcover_class_palette').getInfo()
print(class_palette)

In [ ]:
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(
    ['remapped'], ['landcover']
)
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

In [ ]:
Map.addLayer(nlcd, {}, 'NLCD')
Map

#### Make training data

In [ ]:
# Make the training dataset.
points = nlcd.sample(
    **{
        'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, 'training', False)

In [ ]:
print(points.size().getInfo())

In [ ]:
print(points.first().getInfo())

#### Split training and testing

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
sample = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Adds a column of deterministic pseudorandom numbers.
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [ ]:
training.first().getInfo()

In [ ]:
validation.first().getInfo()

#### Train the classifier

In [ ]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

#### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

#### Render categorical map

In [ ]:
class_values = nlcd.get('landcover_class_values').getInfo()
print(class_values)

In [ ]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
print(class_palette)

In [ ]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Land cover')
Map

#### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

#### Add a legend to the map

In [ ]:
Map.add_legend(builtin_legend='NLCD')
Map

#### Accuracy assessment

##### Training dataset

In [ ]:
train_accuracy = classifier.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

##### Validation dataset

In [ ]:
validated = validation.classify(classifier)

In [ ]:
validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

#### Download confusion matrix

In [ ]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

#### Reclassify land cover map

In [ ]:
landcover = landcover.remap(new_class_values, raw_class_values).select(
    ['remapped'], ['classification']
)

In [ ]:
landcover = landcover.set('classification_class_values', raw_class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Final land cover')
Map

#### Export the result

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

In [ ]:
geemap.ee_export_image_to_drive(
    landcover, description='landcover', folder='export', scale=900
)

## Crearting training samples

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

In [ ]:
if Map.user_rois is not None:
    training_samples = Map.user_rois
    print(training_samples.getInfo())

## Using locally trained machine learning models

In [ ]:
import ee
import geemap
import pandas as pd

from geemap import ml
from sklearn import ensemble

In [ ]:
geemap.ee_initialize()

### Train a model locally using scikit-learn

In [ ]:
# read the feature table to train our RandomForest model
# data taken from ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

url = "https://raw.githubusercontent.com/giswqs/geemap/master/examples/data/rf_example.csv"
df = pd.read_csv(url)

In [ ]:
df

In [ ]:
# specify the names of the features (i.e. band names) and label
# feature names used to extract out features and define what bands

feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = "landcover"

In [ ]:
# get the features and labels into separate variables
X = df[feature_names]
y = df[label]

In [ ]:
# create a classifier and fit
n_trees = 10
rf = ensemble.RandomForestClassifier(n_trees).fit(X, y)

### Convert a sklearn classifier object to a list of strings

In [ ]:
# convert the estimator into a list of strings
# this function also works with the ensemble.ExtraTrees estimator
trees = ml.rf_to_strings(rf, feature_names)

In [ ]:
# print the first tree to see the result
print(trees[0])

In [ ]:
print(trees[1])

In [ ]:
# number of trees we converted should equal the number of trees we defined for the model
len(trees) == n_trees

### Convert sklearn classifier to GEE classifier

In [ ]:
# create a ee classifier to use with ee objects from the trees
ee_classifier = ml.strings_to_classifier(trees)

In [ ]:
# ee_classifier.getInfo()

### Classify image using GEE classifier

In [ ]:
# Make a cloud-free Landsat 8 TOA composite (from raw imagery).
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(
    collection=l8.filterDate('2018-01-01', '2018-12-31'), asFloat=True
)

In [ ]:
# classify the image using the classifier we created from the local training
# note: here we select the feature_names from the image that way the classifier knows which bands to use
classified = image.select(feature_names).classify(ee_classifier)

In [ ]:
# display results
Map = geemap.Map(center=(37.75, -122.25), zoom=11)

Map.addLayer(
    image,
    {"bands": ['B7', 'B5', 'B3'], "min": 0.05, "max": 0.55, "gamma": 1.5},
    'image',
)
Map.addLayer(
    classified,
    {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},
    'classification',
)

Map

### Save trees to the cloud

In [ ]:
user_id = geemap.ee_user_id()
user_id

In [ ]:
# specify asset id where to save trees
# be sure to change <user_name> to your ee user name
asset_id = user_id + "/random_forest_strings_test"
asset_id

In [ ]:
# kick off an export process so it will be saved to the ee asset
ml.export_trees_to_fc(trees, asset_id)

# this will kick off an export task, so wait a few minutes before moving on

In [ ]:
# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [ ]:
# display results
# we should get the exact same results as before
Map = geemap.Map(center=(37.75, -122.25), zoom=11)

Map.addLayer(
    image,
    {"bands": ['B7', 'B5', 'B3'], "min": 0.05, "max": 0.55, "gamma": 1.5},
    'image',
)
Map.addLayer(
    classified,
    {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},
    'classification',
)

Map

### Save trees locally

In [ ]:
import os

out_csv = os.path.expanduser("~/Downloads/trees.csv")

In [ ]:
ml.trees_to_csv(trees, out_csv)

In [ ]:
another_classifier = ml.csv_to_classifier(out_csv)

In [ ]:
classified = image.select(feature_names).classify(another_classifier)

In [ ]:
# display results
# we should get the exact same results as before
Map = geemap.Map(center=(37.75, -122.25), zoom=11)

Map.addLayer(
    image,
    {"bands": ['B7', 'B5', 'B3'], "min": 0.05, "max": 0.55, "gamma": 1.5},
    'image',
)
Map.addLayer(
    classified,
    {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},
    'classification',
)

Map

## Timeseries analysis

### Quality mosaic

### Import libraries

In [ ]:
import ee
import geemap

### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

### Define a region of interest (ROI)

In [ ]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'coutries')

In [ ]:
roi = countries.filter(ee.Filter.eq('id', 'USA'))
Map.addLayer(roi, {}, 'roi')

### Filter ImageCollection

In [ ]:
start_date = '2019-01-01'
end_date = '2019-12-31'

l8 = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
    .filterBounds(roi)
    .filterDate(start_date, end_date)
)

In [ ]:
# print(l8.size().getInfo())

### Create a median composite

In [ ]:
median = l8.median()

visParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')

### Define functions to add time bands

In [ ]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [ ]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [ ]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

### Map over an ImageCollection

In [ ]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

### Create a quality mosaic

In [ ]:
greenest = withNDVI.qualityMosaic('NDVI')

In [ ]:
greenest.bandNames().getInfo()

### Display the max value band

In [ ]:
ndvi = greenest.select('NDVI')
palette = [
    '#d73027',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#d9ef8b',
    '#a6d96a',
    '#66bd63',
    '#1a9850',
]
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [ ]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

### Display time bands

In [ ]:
Map.addLayer(
    greenest.select('month'),
    {'palette': ['red', 'blue'], 'min': 1, 'max': 12},
    'Greenest month',
)

In [ ]:
Map.addLayer(
    greenest.select('doy'),
    {'palette': ['brown', 'green'], 'min': 1, 'max': 365},
    'Greenest doy',
)

## Interactive charts

In [ ]:
import ee
import geemap
import geemap.chart as chart

In [ ]:
# geemap.update_package()

### Creating a chart from ee.FeatureCollection by feature

In [ ]:
Map = geemap.Map()

features = ee.FeatureCollection('projects/google/charts_feature_example').select(
    '[0-9][0-9]_tmean|label'
)

Map.addLayer(features, {}, "Ecoregions")
Map

In [ ]:
df = geemap.ee_to_pandas(features)
df

In [ ]:
xProperty = "label"
yProperties = [str(x).zfill(2) + "_tmean" for x in range(1, 13)]

labels = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec',
]
colors = [
    '#604791',
    '#1d6b99',
    '#39a8a7',
    '#0f8755',
    '#76b349',
    '#f0af07',
    '#e37d05',
    '#cf513e',
    '#96356f',
    '#724173',
    '#9c4f97',
    '#696969',
]
title = "Average Monthly Temperature by Ecoregion"
xlabel = "Ecoregion"
ylabel = "Temperature"

In [ ]:
options = {
    "labels": labels,
    "colors": colors,
    "title": title,
    "xlabel": xlabel,
    "ylabel": ylabel,
    "legend_location": "top-left",
    "height": "500px",
}

In [ ]:
chart.feature_byFeature(features, xProperty, yProperties, **options)

### Creating a chart from ee.FeatureCollection by property

In [ ]:
Map = geemap.Map()

features = ee.FeatureCollection('projects/google/charts_feature_example').select(
    '[0-9][0-9]_ppt|label'
)

Map.addLayer(features, {}, 'Features')
Map

In [ ]:
df = geemap.ee_to_pandas(features)
df

In [ ]:
keys = [str(x).zfill(2) + "_ppt" for x in range(1, 13)]
values = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec',
]

In [ ]:
xProperties = dict(zip(keys, values))
seriesProperty = "label"

In [ ]:
options = {
    'title': "Average Ecoregion Precipitation by Month",
    'colors': ['#f0af07', '#0f8755', '#76b349'],
    'xlabel': "Month",
    'ylabel': "Precipitation (mm)",
    'legend_location': "top-left",
    "height": "500px",
}

In [ ]:
chart.feature_byProperty(features, xProperties, seriesProperty, **options)

### Histogram

In [ ]:
import ee
import geemap
import geemap.chart as chart

In [ ]:
geemap.ee_initialize()

In [ ]:
source = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m').toBands()
region = ee.Geometry.Rectangle(-123.41, 40.43, -116.38, 45.14)
my_sample = source.sample(region, 5000)
property = '07_ppt'

In [ ]:
options = {
    "title": 'July Precipitation Distribution for NW USA',
    "xlabel": 'Precipitation (mm)',
    "ylabel": 'Pixel count',
    "colors": ['#1d6b99'],
}

In [ ]:
chart.feature_histogram(my_sample, property, **options)

In [ ]:
chart.feature_histogram(my_sample, property, maxBuckets=30, **options)

In [ ]:
chart.feature_histogram(my_sample, property, minBucketWidth=0.5, **options)

In [ ]:
chart.feature_histogram(my_sample, property, minBucketWidth=3, maxBuckets=30, **options)

## Global land cover area

In [ ]:
Map = geemap.Map()
dataset = ee.ImageCollection("ESA/WorldCover/v100").first()
Map.addLayer(dataset, {'bands': ['Map']}, 'ESA Land Cover')
Map.add_legend(builtin_legend='ESA_WorldCover')
Map

In [ ]:
df = geemap.image_area_by_group(
    dataset, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
df

In [ ]:
df.to_csv('esa_area.csv')

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap('HYBRID')

nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = nlcd.select('landcover')

Map.addLayer(landcover, {}, 'NLCD Land Cover 2019')
Map.add_legend(
    title="NLCD Land Cover Classification", builtin_legend='NLCD', height='465px'
)
Map

In [ ]:
df = geemap.image_area_by_group(
    landcover, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
df

In [ ]:
df.to_csv('nlcd_area.csv')

## Whitebox-Tools

In [ ]:
import os
import geemap
import whiteboxgui

In [ ]:
out_dir = os.path.expanduser('~/Downloads')
dem = os.path.join(out_dir, 'dem.tif')

if not os.path.exists(dem):
    dem_url = 'https://drive.google.com/file/d/1vRkAWQYsLWCi6vcTMk8vLxoXMFbdMFn8/view?usp=sharing'
    geemap.download_from_gdrive(dem_url, 'dem.tif', out_dir, unzip=False)

In [ ]:
Map = geemap.Map()
Map

In [ ]:
whiteboxgui.show()

In [ ]:
whiteboxgui.show(tree=True)

## Summary

## References